<a href="https://colab.research.google.com/github/FazilShah/Machine-Learning/blob/master/Neural_Netwrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy import optimize

In [ ]:
def hyp(z):
  return 1/(1+np.exp(-z))

In [ ]:
def gradsigmoid(z):
  return np.multiply(hyp(z),(1-hyp(z)))

In [ ]:
def rand_param(a,b,epsilon):
  return (np.random.rand(a,b+1)*2*epsilon) - epsilon

In [ ]:
Data = pd.ExcelFile('ex3d1.xlsx')
Theta1 = pd.read_excel(Data, 'theta1', header= None)
Theta2 = pd.read_excel(Data, 'theta2', header=None)
X = pd.read_excel(Data, 'X', header=None)
Y = pd.read_excel(Data, 'y', header=None)

In [ ]:
def nnCostFunction(nn_params ,*args):
  a1 = 0
  a2 = 0
  input_layer_size, hidden_layer_size,num_labels,X, Y,lamda = args 
  theta1 = np.reshape(nn_params[:hidden_layer_size*(input_layer_size+1)], (hidden_layer_size, input_layer_size+1), 'F')
  theta2 = np.reshape(nn_params[hidden_layer_size*(input_layer_size+1):], (num_labels, hidden_layer_size+1), 'F')
  theta1 = np.asarray(theta1)
  theta2 = np.asarray(theta2)
  Theta1_grad = np.zeros((hidden_layer_size,input_layer_size+1))
  Theta2_grad = np.zeros((num_labels,hidden_layer_size+1))
  #this part relabels Y as (m,k) matrix where K = no. of lables & m = training examples.
  Ind = np.eye(num_labels)
  new_Y = np.zeros((len(Y),num_labels), dtype=int)
  for i in range(len(Y)):
    index = Y.loc[i] - 1
    new_Y[i,:] = Ind[index,:]
  #this part does the feedforward implementation to compute error

  a1 = X
  z2 = np.matmul(np.asarray(a1),np.asarray(np.transpose(theta1)))
  a2 = hyp(z2)
  a2 = np.insert(a2,0,1,axis=1)
  z3 = np.matmul(np.asarray(a2), np.asarray(np.transpose(theta2)))
  a3 = hyp(z3)

  cost_vec = np.multiply((-new_Y),np.log(a3)) - np.multiply((1-new_Y),np.log(1-a3))

  cost = np.sum(cost_vec,axis=1)
  J = (1/len(Y))*sum(cost)
  

  #This part calculates the cost of regularized
  theta1nobias = theta1[:,1:]
  theta2nobias = theta2[:,1:]
  reg = np.sum(np.square(theta1nobias)) + np.sum(np.square(theta2nobias))
  reg2 = (lamda*reg)/(2*len(Y))
  J+=reg2
  return J
  #This part calculates Theta1_grad and Theta2_grad
def gradients(nn_params ,*args):
  Delta1 = 0
  Delta2 = 0
  input_layer_size, hidden_layer_size,num_labels,X, Y,lamda = args 
  theta1 = np.reshape(nn_params[:hidden_layer_size*(input_layer_size+1)], (hidden_layer_size, input_layer_size+1), 'F')
  theta2 = np.reshape(nn_params[hidden_layer_size*(input_layer_size+1):], (num_labels, hidden_layer_size+1), 'F')
  theta1nobias = theta1[:,1:]
  theta2nobias = theta2[:,1:]
  Ind = np.eye(num_labels)
  new_Y = np.zeros((len(Y),num_labels), dtype=int)
  for i in range(len(Y)):
    index = Y.loc[i] - 1
    new_Y[i,:] = Ind[index,:]
  for i in range(len(Y)):
    a11 = np.asarray(X.loc[i,:])
    a11 = np.reshape(a11,(input_layer_size+1,1))
    z22 = np.matmul(theta1,a11)
    a22 = np.asarray(hyp(z22))
    a22 = np.insert(a22,0,1)
    z33 = np.matmul(theta2,a22)
    a33 = np.asarray(hyp(z33))

    #now we will calculate errors in each layer (layer size = 3)

    d3 = a33 - new_Y[i,:]
    d21 = np.matmul(np.transpose(theta2nobias),d3)
    d2 = np.multiply(np.reshape(d21,(hidden_layer_size,1)), gradsigmoid(z22))
    d3c = np.reshape(d3,(num_labels,1))
    a2c = np.reshape(a22,(hidden_layer_size+1,1)) 
    Delta1 = Delta1 +  np.matmul(d2,np.transpose(a11))
    Delta2 = Delta2 + np.matmul(d3c, np.transpose(a2c))
  
  Theta1_grad = (1/(len(Y)))*Delta1
  Theta2_grad = (1/(len(Y)))*Delta2

  Theta1_grad[:,1:] = Theta1_grad[:,1:] + (lamda/len(Y))*theta1nobias
  Theta2_grad[:,1:] = Theta2_grad[:,1:] + (lamda/len(Y))*theta2nobias

  return  np.hstack((Theta1_grad.ravel(order='F'), Theta2_grad.ravel(order='F')))

In [ ]:
initial_theta1 = rand_param(25,400,0.12)
initial_theta2 = rand_param(10,25,0.12)
nn_initial_params = np.hstack((initial_theta1.ravel(order='F'), initial_theta2.ravel(order='F')))


In [ ]:
X.insert(0,'0',1)

In [ ]:
args = (400,25,10,X,Y,1)

In [ ]:
res1 = optimize.fmin_cg(nnCostFunction,nn_initial_params,gradients,args,maxiter=50)

In [ ]:
def Gradient_check(params, *args):
  nparams = params.flatten()
  input_layer_size, hidden_layer_size,num_labels,X, Y,lamda = args
  esp = 0.0001
  for i in range(10):
    x = int(np.random.rand()*len(nparams))
    diff = np.zeros((len(params),1))
    print(x)
    diff[x] = esp
    low_cost = nnCostFunction(nparams-diff.flatten(), *args)
    high_cost = nnCostFunction(nparams+diff.flatten(), *args)
    num_grad = (high_cost - low_cost)/(2*esp)
    print(num_grad)
    print(gradients(nparams,400,25,10,X,Y,1)[x])



In [ ]:
Gradient_check(nn_initial_params, 400,25,10,X,Y,1)

5532
3.762274936036647e-06
3.7622563228988317e-06
5802
-0.01733303173878653
-0.01733303158855285
5797
-0.008358149190534903
-0.008358149078466411
7630
0.005927289623564036
0.005927289627060342
8442
4.627153327163569e-05
4.627145944229761e-05
2921
0.0008126048411938314
0.0008126047980264244
6103
0.0014218748312089247
0.0014218749936164912
2124
0.00016642240918685047
0.0001664224311328214
8434
-0.00018892495123878916
-0.00018892491022800325
4722
-0.0037031654853691975
-0.0037031656515665577
